# Inversiones por estado


Este notebook tiene como finalidad poder sacar el total de las inversiones realizadas por estado

## Paso a paso

* **Clasificar los anuncios por marca**: Con esto obtendremos de igual manera, a qué marca pertenece el anuncio según la plataforma
* **Costos totales por plataformas**: Se busca tener el total de costos agrupando por Marca y Region. Este archivo se genera dentro del archivo **invest_per_state/rql_data.ipynb** según un rango de fechas dado
* RQLs generados: Esta parte busca poder 
* **Combinar datos de Geolocalización entre plataformas**: Con el fin de poder hacer cruces entre plataformas de manera mas sencilla, se usa una tabla que tiene información de nombres de regiones / estados con sus equivalentes
* **Costos totales entre Meta y GAds por estados**: Acá se saca el total de inversión por estados entre las plataformas

In [1]:
import sys
sys.path.append('../')
import pandas as pd
from utils.classify_ads import *
from utils.df_utils import set_week_and_year

## Clasificar los anuncios por marca

In [2]:
# Lectura del archivo de Meta
df_meta_ads = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\BD Ads\Abril\14042025\MX BD Ads saturación mensual - Meta.csv")

# Aplicar la función a la columna Ad name
df_meta_ads['Marca'] = df_meta_ads['Ad name'].apply(classify_ads)

# Agregar columnas con el número de semana y año
df_meta_ads = set_week_and_year(df_meta_ads)

In [3]:
# Lectura del archivo de Google Ads
df_google_ads = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\BD Ads\Abril\14042025\MX BD Ads saturación mensual - GAds.csv")

# Aplicar la función a la columna Ad name
df_google_ads['Marca'] = df_google_ads['Ad group name'].apply(classify_ads)

# Agregar columnas con el número de semana y año
df_google_ads = set_week_and_year(df_google_ads)

## Costo totales por plataformas

Acá se busca sacar el costo total por Marca, región y semana

#### Meta

In [4]:
df_meta_invest_per_state = df_meta_ads.groupby(['Marca', 'Region', 'Semana', 'Año'])['Cost'].sum().reset_index()

In [ ]:
df_meta_invest_per_state

### Google Ads

In [6]:
df_google_ads_invest_per_state = df_google_ads.groupby(['Marca', 'Region', 'Semana', 'Año'])['Cost'].sum().reset_index()

In [ ]:
df_google_ads_invest_per_state

## RQLs generados

Costos totales, incluye PureBrand y referidos

In [ ]:
rqls_por_marca = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\invest_per_state\outputs\RQLs Amplitude\20250414-rql_resumen_per_brand.csv")

# Semana a consultar
# rqls_por_marca.drop(columns=["state", "day"], inplace=True)
# rqls_por_marca = rqls_por_marca.groupby(['brand', 'Semana', 'Año'])['rql'].sum().reset_index()
rqls_por_marca

In [ ]:
total_rqls = rqls_por_marca["rql"].sum()
print(f"Total de RQLs para la semana: {total_rqls}")

In [13]:
week = 15

Selección de semana a consultar en cada una de las plataformas y archivo de RQLs (aunque ya viene en la semana a consultar)

In [14]:
df_meta_invest_per_state_week = df_meta_invest_per_state[df_meta_invest_per_state["Semana"] == week]
df_google_ads_invest_per_state_week = df_google_ads_invest_per_state[df_google_ads_invest_per_state["Semana"] == week]
rqls_por_marca_week = rqls_por_marca[rqls_por_marca["Semana"] == week]

In [ ]:
df_google_ads_invest_per_state_week

## Combinar datos de Geolocalización entre plataformas

Esto sirve para poder hacer la combinación de datos entre las plataformas ya que cada plataforma maneja nombres distintos entre regiones/estados. Este archivo tiene el "equivalente" de cada nombre entre plataforma con uno base "amplitude_ips"

In [16]:
df_state_per_media = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\Geolocalización\MX equivalencias geo regiones - Equivalencias_Medios.csv")

In [ ]:
df_state_per_media.head(2)

### Combinar con plataformas

#### Meta

In [18]:
df_rql_per_state_and_brand_merged_meta = pd.merge(df_meta_invest_per_state_week, df_state_per_media[["Amplitude_ips", "Meta"]], 
                                             left_on="Region", 
                                             right_on="Meta", 
                                             how = "left")

#### GAds

In [19]:
df_rql_per_state_and_brand_merged_gads = pd.merge(df_google_ads_invest_per_state_week, df_state_per_media[["Amplitude_ips", "GAds"]], 
                                             left_on="Region", 
                                             right_on="GAds", 
                                             how = "left")

In [ ]:
df_rql_per_state_and_brand_merged_gads

In [21]:
asd = df_rql_per_state_and_brand_merged_gads[df_rql_per_state_and_brand_merged_gads["Semana"] == 15]

Para diferenciar los costos al hacer el merge

In [23]:
df_rql_per_state_and_brand_merged_meta.rename(columns={"Cost": "cost_meta"}, inplace=True)
df_rql_per_state_and_brand_merged_gads.rename(columns={"Cost": "cost_gads"}, inplace=True)

In [24]:
def add_totals(valores):
    total_row = pd.Series(valores)
    return pd.DataFrame([total_row])

### Costos entre GAds y Meta

Se unen ambos DF para ver como han sido los costos por estados

In [ ]:
df_merged = pd.merge(df_rql_per_state_and_brand_merged_meta[["Marca", "Amplitude_ips", "cost_meta"]], 
                     df_rql_per_state_and_brand_merged_gads[["Marca", "Amplitude_ips", "cost_gads"]], 
                     on = ["Marca", "Amplitude_ips"], 
                     how = "outer") # Ya que puede que en Meta no hayan estados (amplitude_ips) que GAds si contenga
df_merged["cost_meta"].fillna(0, inplace=True)
df_merged["cost_gads"].fillna(0, inplace=True)
df_merged

## Inversión total entre Meta y GAds por estados

#### Por estados

In [27]:
per_state = df_merged.groupby(["Amplitude_ips"])[["cost_meta", "cost_gads"]].sum().reset_index()
costo_total_meta = int(df_merged["cost_meta"].sum())
costo_total_gads = int(df_merged["cost_gads"].sum())
total_cost = int(costo_total_meta + costo_total_gads)

df_total = add_totals({"Amplitude_ips": "Total", "cost_meta": costo_total_meta, "cost_gads": costo_total_gads})
per_state_with_totals = pd.concat([per_state, df_total]) 

per_state_with_totals["cost_meta"] = per_state_with_totals["cost_meta"].astype(int) 
per_state_with_totals["cost_gads"] = per_state_with_totals["cost_gads"].astype(int) 


# display(per_state_with_totals)


# print(f"Costo total Meta: {costo_total_meta:,.2f} MXN")
# print(f"Costo total GAds: {costo_total_gads:,.2f} MXN")
# print(f"Costo total Meta + GAds: {total_cost:,.2f} MXN")

#### Por marcas

In [28]:
per_brand = df_merged.groupby(["Marca"])[["cost_meta", "cost_gads"]].sum().reset_index()
costo_total_meta_per_brand = int(df_merged["cost_meta"].sum())
costo_total_gads_per_brand = int(df_merged["cost_gads"].sum())
total_cost_per_brand = int(costo_total_meta + costo_total_gads)


df_total = add_totals({"Marca": "Total", "cost_meta": costo_total_meta_per_brand, "cost_gads": costo_total_gads_per_brand})
per_brand_with_totals = pd.concat([per_brand, df_total]) 

per_brand_with_totals["cost_meta"] = per_brand_with_totals["cost_meta"].astype(int) 
per_brand_with_totals["cost_gads"] = per_brand_with_totals["cost_gads"].astype(int) 

# display(per_brand_with_totals)


# print(f"Costo total Meta: {costo_total_meta_per_brand:,.2f} MXN")
# print(f"Costo total GAds: {costo_total_gads_per_brand:,.2f} MXN")
# print(f"Costo total Meta + GAds: {total_cost_per_brand:,.2f} MXN")

## RQLs obtenidos por estados

In [65]:
rqls_per_state = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\invest_per_state\RQLs Amplitude\20250414-rql_resumen_per_state.csv")
rqls_per_state = rqls_per_state.drop(columns = ["Semana", "Año"])

In [66]:
rql_sum_rql_per_state = int(rqls_per_state["rql"].sum())

row_total = add_totals({'state': "Total", "rql": rql_sum_rql_per_state})

rqls_per_state_with_totals = pd.concat([rqls_per_state, row_total])
rqls_per_state_with_totals["rql"] = rqls_per_state_with_totals["rql"].astype(int) 

### trash

In [ ]:
# rql_total = int(rqls_per_state["rql"].sum())
# rql_total


# ---

# rqls_por_marca_week_copy = rqls_por_marca_week.copy()
# rqls_por_marca_week = rqls_por_marca_week_copy.groupby(["brand", "Meta", "GAds", "Semana", "Año"])["rql"].sum().reset_index()
# rqls_por_marca_week

# ----

# rqls_por_marca_week_total = rqls_por_marca_week["rql"].sum()
# print(rqls_por_marca_week_total)

# ----


# df_meta_cost_invest_rql_per_state = pd.merge(df_meta_invest_per_state_week, rqls_por_marca_week[["brand", "Meta", "rql"]],
#                                             left_on = ["Region", "Marca"],
#                                             right_on = ["Meta", "brand"],
#                                             how="left")
# df_meta_cost_invest_rql_per_state = df_meta_cost_invest_rql_per_state[["Semana", "Año", "Marca", "brand", "Region", "Cost", "Meta", "rql"]]


# ----

# df_meta_cost_invest_rql_per_state.fillna({"rql": 0}, inplace=True)
# df_meta_cost_invest_rql_per_state

# ----

# df_meta_marcas_no_generaron_rql = df_meta_cost_invest_rql_per_state.copy()
# df_meta_marcas_no_generaron_rql = df_meta_marcas_no_generaron_rql[df_meta_marcas_no_generaron_rql["rql"] == 0]
# df_meta_marcas_no_generaron_rql.fillna({"rql": 0}, inplace=True)
# df_meta_marcas_no_generaron_rql.head(2)


# -----

# df_meta_cost_invest_rql_per_state_invertidos =  df_meta_cost_invest_rql_per_state[df_meta_cost_invest_rql_per_state["Meta"].notna()]
# df_meta_cost_invest_rql_per_state_invertidos


# ----


# rql_meta = df_meta_cost_invest_rql_per_state_invertidos["rql"].sum()
# print(f"RQLs Meta: {rql_meta}")

# -----

# df_gads_cost_invest_rql_per_state = pd.merge(df_google_ads_invest_per_state_week, rqls_por_marca_week[["brand", "GAds", "rql"]],
#                                             left_on = ["Region", "Marca"],
#                                             right_on = ["GAds", "brand"],
#                                             how="left")
# df_gads_cost_invest_rql_per_state = df_gads_cost_invest_rql_per_state[["Semana", "Año", "Marca", "brand", "Region", "Cost", "GAds", "rql"]]

# ----

# df_gads_cost_invest_rql_per_state_invertidos = df_gads_cost_invest_rql_per_state[df_gads_cost_invest_rql_per_state["GAds"].notna()]
# df_gads_cost_invest_rql_per_state_invertidos.head(2)

# ----
# asd = rqls_por_marca_week["rql"].sum()
# print(f"asd: {asd}")


# ----


# df_gads_rqls_generados = df_gads_cost_invest_rql_per_state_invertidos["rql"].sum()
# print(f"df_gads_rqls_generados: {df_gads_rqls_generados}")

# ----

# df_meta_rqls_generados = df_meta_cost_invest_rql_per_state_invertidos["rql"].sum()
# print(f"df_meta_rqls_generados: {df_meta_rqls_generados}")

# ----

# total_rqls = df_meta_rqls_generados + df_gads_rqls_generados
# print(f"Total de RQLs generados: {total_rqls}")

## RQLs obtenidos por marcas

In [33]:
rqls_por_marca_week_copy = rqls_por_marca_week.copy()
rqls_por_marca_week = rqls_por_marca_week_copy.groupby(["brand"])["rql"].sum().reset_index()
rqls_por_marca_week.sort_values(by="rql", ascending=False, inplace=True)


rqls_per_week = int(rqls_por_marca_week["rql"].sum())
rqls_per_week

rqls_per_week_with_totals = add_totals({'brand': "Total", "rql": rqls_per_week})

rqls_por_marca_week_with_totals = pd.concat([rqls_por_marca_week, rqls_per_week_with_totals])
rqls_por_marca_week_with_totals["rql"] = rqls_por_marca_week_with_totals["rql"].astype(int) 

## RQLs obtenidos por Región

## Reporte

#### Inversión por marcas

In [67]:
per_brand_with_totals

,Marca,cost_meta,cost_gads
0,Bajaj,133682,21178
1,Dinamo,0,773
2,Generica,127901,76825
3,Honda,7676,10166
4,Italika,8055,1635
5,Suzuki,6918,1675
6,TVS,6113,3549
7,Treck,0,160
8,Vento,8810,4904
9,Yamaha,10537,3383


#### Inversión por estado

In [68]:
per_state_with_totals

,Amplitude_ips,cost_meta,cost_gads
0,Aguascalientes,28,0
1,Baja California,113,0
2,Baja California Sur,10,0
3,Campeche,88,436
4,Chiapas,22,0
5,Chihuahua,18,0
6,Coahuila,447,92
7,Colima,70,0
8,Durango,26,0
9,Guanajuato,238,0


RQLs por marcas

In [69]:
rqls_por_marca_week_with_totals

,brand,rql
0,Bajaj,1927
19,Vento,625
2,CF Moto,507
9,Italika,368
7,Honda,315
20,Yamaha,194
16,Suzuki,192
11,KTM,191
3,CFLITE,112
17,TVS,112


RQLs por estados

In [70]:
rqls_per_state_with_totals

,state,rql
0,Aguascalientes,14
1,Baja California,9
2,Baja California Sur,8
3,Campeche,39
4,Chihuahua,15
5,Ciudad de México,855
6,Coahuila de Zaragoza,24
7,Colima,26
8,Durango,6
9,Estado de México,4


## Exportaciones

In [ ]:
actual_date = pd.Timestamp.now().strftime('%Y%m%d')
print(actual_date)

In [40]:
per_brand_with_totals.to_csv(f"outputs/Totales/{actual_date}-total_cost_per_brand.csv", index = False)

In [41]:
per_state_with_totals.to_csv(f"outputs/Totales/{actual_date}-total_cost_per_state_with_totals.csv", index = False)

In [43]:
rqls_por_marca_week_with_totals.to_csv(f"outputs/Totales/{actual_date}-total_rqls_por_marca_week.csv", index = False)

In [71]:
rqls_per_state_with_totals.to_csv(f"outputs/Totales/{actual_date}-total_rqls_per_state_with_totals.csv", index = False)